In [1]:
import numpy as np
import random
import pandas as pd
import copy
import matplotlib.pyplot as plt

# online plotting
%matplotlib notebook  

# Initialization Phase
- Starting with the Class for our cars
- Reading in the documents
- Initializing a population of solutions

In [2]:
class Car(object):
    # a car is an object with the information about its capacity, cost, 
    # customers/route and route costs.
    def __init__(self, capacity, cost, route, route_cost):
        # capacitiy of this car
        self.capacity = capacity
        # general costs of this car
        self.cost = cost  
        # list of the customers in the beginning or 
        # when 'ordered' later it is the route
        self.route = route    
        # costs of the car's current route
        self.route_cost = route_cost  
        # Note: every time the order of costumers changes, 
        # the route_cost has to be changed accordingly 
        
    def update_route_and_route_cost(self, route, route_cost):
        # a function which updates the route and route cost
        self.route = route
        self.route_cost = route_cost

In [3]:
#Read the documents and create a list of cars with the the lines

def chunks(l, n):
    # helper function for reading in our data
    # creates chunks of a specific size
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

with open('capacity.txt','r') as text:
    capacity_read = text.read()
    capacity = [int(i) for i in capacity_read.split()]

with open('demand.txt','r') as text:
    demand_read = text.read()
    demands = [int(j) for j in demand_read.split()]
    demands = [0] + demands
    
with open('transportation_cost.txt', 'r') as text:
    trans_cost_read = text.read()
    trans_cost = [int(k) for k in trans_cost_read.split()]

with open('distance.txt', 'r') as text:
    distance_read = text.read()
    distance = [int(l) for l in distance_read.split()]
distance = list(chunks(distance, int(np.sqrt(len(distance)))))


# Optimizing the individual cars' routes after the Initialization using ACO
This is the Ant part for the Vehicle Routing Problem 
Each car got assigned some amount of customers and with those, we calculate a route and improve this route with the ant algorithm. In the End we save the total costs for each car. 
How we did it: 
- we used the pheromone matrix and set every customer -10 that was not assigned to the car in order to avoid that the ants go over that path 
- the we applied the TSP on the rest to get the shortest route
- we saved the route in best_fitness[0]
- we updated the costs of the total transportation for each car with the following equation: car_cost* shortest route
- we saved the order of the route in the end for each car 

In [4]:
# Initialization of the ACO. 
def initialization_ACO(ph_value, dist,customers):
    # ph_value= initial pheromone value
    # dist = distance matrix
    # pheromone_mat = pheromone matrix
    # heuristic_matrix = inverse of the distance matrix

    pheromone_mat = np.full((len(dist), len(dist)), ph_value) # create matrix, fill it with a specific number
    #np.fill_diagonal(pheromone_mat, -10) #
    #we define what ways can be used (between the customers)
    #for customer in customers:
     #   #pheromone_mat[0][customer]= ph_value
      #  for nextcustomer in customers: 
       #     if nextcustomer != customer: 
        #        pheromone_mat[customer][nextcustomer] = ph_value
    
    
    
    #inverse of the distance matrix
    heuristic_matrix = 1/np.array(dist) #function that gives the inverse of a matrix

    return pheromone_mat, heuristic_matrix

In [5]:
def ant_probability(current_city, z, pheromone_mat, h_mat, alpha, beta):
    # the current_city refers to our ant's location
    # z is the list of cities left to pool from for our ant
    # this function is an implementation of the probability function
    # p_ij = (t_ij^alpha * n_ij^beta)/sum(t_iz^alpha*n_iz^beta)
    
    probabilities = []
    pheromones = np.array([])
    heuristics = np.array([])

    # we first create all t_iz^alpha (pheromones) and all n_iz^beta (heuristics)
    for city in z:
        pheromones = np.append(pheromones, pheromone_mat[current_city, city]**alpha)
        heuristics = np.append(heuristics, h_mat[current_city, city]**beta)
    
    # then we compute the denominator
    denominator = np.matmul(pheromones, heuristics)
    
    # afterwards we compute the probability of each city left in our set z = Z[a]
    
    for city in z:
        city_index = z.index(city)
        probabilities.append(pheromones[city_index]*heuristics[city_index]/denominator)
    
    return probabilities  # the list of probabilities for choosing each city in z

In [6]:
#Roulette Wheel selection   
#return the index of randomly chosen city
def roulette_wheel_ACO(probabilities):
        
    #For accumulating the probability for later use
    previous_probability = 0.0
    cumulative_probability = probabilities.copy()
    for i, p in enumerate(probabilities):
        cumulative_probability[i] = previous_probability + cumulative_probability[i]
        previous_probability = cumulative_probability[i]
    #Choosing probabilitistically randomly 
    rand = random.uniform(0, 1)
    
    for index in range(len(cumulative_probability)):
        if rand < cumulative_probability[index]: 
            break
        
    return index

In [7]:
def roulette_wheel_BCO(costs):
    fitnesses = 1 / np.array(costs) # fitness of the solution, longer distances ==> worse solution
    normalized_fits = fitnesses / np.sum(fitnesses)
    bins = np.cumsum(normalized_fits)# returns an array of same length. At each spot is gives the sum of all alements before added up
    roulette_value = [np.random.uniform(0,1)] # draw random value between 0 and 1 
    index_next_node = np.digitize(roulette_value, bins)[0]  # states to which bin the value belongs
    return(index_next_node)

In [8]:
def solution_construction(A, dist, pheromone_mat, h_mat, alpha, beta, customers, mode='roulette_wheel'):
    # A is the number of ants
    # dist is the distance matrix
    # pheromone_mat is the current pheromone matrix
    # h_mat is the heuristic matrix (h_ij = 1/dist_ij)
    # alpha for weighting the pheromone value
    # beta for weighting the heuristic value, e.g. the city distance
    # mode determines the mode used to choose the city given the probabilities
    # customers is the list of customers which need to be served
    
    S = customers  # list of cities
    M = [list([]) for i in range(A)]  # the tours of all A ants
   
    for a in range(A):
        z = copy.copy(S[1:-1])  # a copy of our city list for an ant to pool from
        
        # each ant gets its starting city assigned
        
        M[a].append(0)   #starting at the depot
        
        
        # create tours
        while z:
            current_city = M[a][-1]  # look up the current city
            city_probabilities = ant_probability(current_city,z,pheromone_mat, h_mat, alpha, beta)
            if mode == 'max':
                # chose next city by maximum probability value
                next_city_index = city_probabilities.index(max(city_probabilities))
            if mode == 'roulette_wheel':
                # chose next city using the roulette wheel
                next_city_index = roulette_wheel_ACO(city_probabilities)
            
            next_city = z[next_city_index]
            M[a].append(next_city)
            z.remove(next_city)
        M[a].append(0)
            
    return M  #return a list of the tours of all ants

In [9]:
# pheromone on each path evaporates by a defined factor. Returns updated pheromone matrix.

def evaporator(pher_matrix, ev_parameter=0.2):
    pher_matrix_new = (1-ev_parameter) * pher_matrix
    return pher_matrix_new

In [10]:
def walked_through(a_tour, s, e):
    for i in range(-1, len(a_tour)-1):
        if a_tour[i]==s and a_tour[i+1]==e:
            return True
    return False

# pheromone matrix intensification here
def intensification(pheromones, M, dist_mat, p, delta=0.2, mode='best_ant'):
    if mode == 'best_ant':
        best, index = ants_fitness_best(M, dist_mat)
        return [[pheromones[i][j]+delta if walked_through(M[index], i, j)
                 else pheromones[i][j]
                 for j in range(len(pheromones))] for i in range(len(pheromones))]
    
    if mode == 'all-ants':
        fitnesses = ants_fitness(M, dist_mat)
        result = p * np.sum( np.array([ [ [ 1/fitnesses[idx] if walked_through(M[idx], i, j) 
                                                       else 0 for idx, ant in enumerate(M)] 
                                                      for j in range(len(pheromones))] 
                                                    for i in range(len(pheromones))]) , 2)
        return result


In [11]:
#Returns a list with the fitness of every ant.
def ants_fitness(M, dist_mat):
    fitness = []

    x = 0
    # Goes to every list in M
    while (x < len(M)):
        # starts in -1 so it counts the back to the origin
        y = 0
        temp = 0
        j = M[x]

        # Goes to every list in M[x] 
        while (y < len(j)-1):
            a = j[y]
            b = j[y+1]
            # Add up all paths from the ant's tour
            temp = temp + dist_mat[a][b]
            y += 1 
        fitness.append(temp)
        x += 1

    return fitness


# Returns an integer with the fitness of the best ant.
def ants_fitness_best(M, dist_mat):
    fitness_best = ants_fitness(M, dist_mat)
    index = fitness_best.index(min(fitness_best))
    fitness_best.sort()
    return fitness_best[0], index


In [12]:
# main ACO function


def aco(dist, population):
    
    # testing parameters
    
    A = 25  # number of ants
    iterations = 15 # number of iterations
    ph_value = 0.5  # pheromone matrix initialisation parameter
    ev_parameter = 0.2  # evaporation parameter  ("proportion")
    delta = 0.1    # intensification parameter
    #combinations = 4  # number of mode combinations we try
    dist = distance  # assign problem 1
    alpha= 1
    beta= 0.5
    #pop_size = 10  # number of solutions to create in the initialization phase
    
    for solution in population: 
        
        for car in solution: #for each object in the list
            
            customers = car.route #get the customers
            #customers = list(map(lambda x: x + 1, customers)) #shifting all customers in order to work with the distance matrix since 0 is the depot 
            pher_mat, h_mat = initialization_ACO(ph_value, dist, customers) # create pheromone matrix and heuritic
            #print (pher_mat)
            fitness_list = []
            #execute the algorithm 
            
            for i in range(iterations): #let the ants run 
                M = solution_construction(A, dist, pher_mat, h_mat, alpha, beta, customers, mode='roulette_wheel') # first solution construct
                pheromone_ev = evaporator(pher_mat, ev_parameter) #evaporate
                pher_mat = np.asarray(intensification(pheromone_ev, M, dist, delta, mode='best_ant')) #intesification
                best_fitness, index = ants_fitness_best(M, dist) # calculate fitness function
                fitness_list.append(best_fitness) # best fitness of each iteration is stored in list


            #choses the best of the "best_fitness" values and not just the last one
            best_fitness_per_car = min(fitness_list)
            index_best_car = fitness_list.index(best_fitness_per_car)
            #save the best ant in the object: best ant* car price
            all_costs = car.cost* best_fitness #calculate the costs of the whole way                              
            car.route_cost = all_costs #save in the object
            car.route = M[index] #saves the route of the car
            
        plt.plot(fitness_list)
        plt.show()
        
  # PROBLEM: ANT fitness ist 0 very often  

# Using GA to optimize the individual solutions in general and finding the best solution

In [13]:
# Problem Class / Problem description

class Problem(object):
    
    def __init__(self, distances, capacities, costs, demands, fitnessFunc1):
        
        #distances matrix between customers
        self.distances = distances  
        
        #Vehicles' capacities
        self.capacities = capacities
        
        #Costs of the vehicles
        self.costs = costs
        
        #Customers' demands
        self.demands = demands
        
        #Fitness based on total cost of the whole solution consists of vehicles' routes
        self.fitnessFunc1 = fitnessFunc1
       

                
#cost function for makespan problem

def cost(genome):
    if sum(car.route_cost for car in genome)==0:
        return np.finfo(dtype=np.float64).eps
    return sum(car.route_cost for car in genome)

    

In [14]:
#Initializer having different functions (versions)

class Initializer(object):  
    
    
    def __init__(self, problem, pop_size):
        self.problem = problem
        self.pop_size = pop_size
    
 
    
    #Initializing randomly

    # Main Initialization to create a popupation
    def main_initialization(self):
        # demands is a list of all customers' demands
        # capacity is a list of the available cars' capacities
        # trans_cost denotes each car's costs and must be of same length as capacity
        # pop_size is a integer giving the population size (number of solutions) to initialize

        # Returns the population which is a list of solutions where a solution is a list of Car objects

        population = []
        cargoList = []
        for i in range (self.pop_size):
            
            solution = [Car(self.problem.capacities[car], self.problem.costs[car], [], 0) for car in range(len(self.problem.costs))]
            amounts = [[] for _ in range(len(self.problem.costs))]
            
            customers = copy.deepcopy(self.problem.demands)
            shuffled_indices = random.sample(range(1, len(self.problem.demands)),len(self.problem.demands)-1)
            
            # as long as customers' demands are not fulfilled:
            customer_count = 0
            cars = list(range(len(self.problem.capacities)))
            while any(item != 0 for item in customers):
                # choose a car type at random and get the car's respective capacity
                car_type = np.random.choice(cars)
                cars.remove(car_type)
                car_capacity = self.problem.capacities[car_type]
                # create an empty list to save the car's costumers
                chosen_customer = []
                route_amounts = []
                # create the car's route:
                while car_capacity > 0 and any(item != 0 for item in customers):
                    # assign the next customer to the current car
                    chosen_customer.append(shuffled_indices[customer_count])
                    # check if customer demand is completely covered by car
                    if customers[shuffled_indices[customer_count]] <= car_capacity:
                        # reduce customer demand in copied list and reduce leftover car demand
                        car_capacity -=  customers[shuffled_indices[customer_count]]
                        route_amounts.append(customers[shuffled_indices[customer_count]]) 
                        customers[shuffled_indices[customer_count]] = 0
                        customer_count += 1
                    else:
                        customers[shuffled_indices[customer_count]] -= car_capacity
                        route_amounts.append(car_capacity) 
                        car_capacity = 0

                # create and add this car object to our current solution        
                solution[car_type].route += [0]+chosen_customer+[0]
                amounts[car_type] += [0] + route_amounts + [0]
                for car_type in cars:
                    solution[car_type].route += [0,0]
                    amounts[car_type] += [0,0]
            # add solution to our current population
            population.append(solution)
            cargoList.append(amounts)
        return population, cargoList





In [15]:
# Selection 

class Selector(object):    
    #Roulette Wheel selection   
    
    def roulette_wheel_GA(self, population, amounts, candidatesSize):
        
        
        #Sum the all fitnesses of all individuals
        sum_of_fitness = sum([1/cost(genome) for genome in population])
        
        #For accumulating the probability for later use
        accumulative_probability = 0.0
        probability = np.zeros(len(population))
        for i, genome in enumerate(population):
            probability[i] = accumulative_probability + (cost(genome) / sum_of_fitness)
            accumulative_probability = probability[i]
        
        
        #candidates is a list of indices referring to the selected individuals
        candidates =  [0]*candidatesSize
        
        for i in range(candidatesSize):
            rand = random.uniform(0, 1)
            for j in range(len(probability)):
                if rand < probability[j]: 
                    break
            candidates[i] = j
        
        return [population[idx] for idx in candidates], [amounts[idx] for idx in candidates]
    
  

In [16]:
#Recombiner having different functions (versions)

class Recombiner(object):
    
    
    #parameters needed for recombination in general
    
    def __init__(self, mating_pool_size, problem):
        self.problem = problem
        self.mating_pool_size = mating_pool_size
    
    
    
    #Whether all customers fully covered
    def all_covered(self, genome, amounts):
        cities = list(range(1, len(self.problem.demands)-1))
        for k, city in enumerate(cities):
            summ = 0
            for i, car in enumerate(genome):
                if city in car.route:
                    city_idx = car.route.index(city)
                    summ += amounts[i][city_idx]
            if summ != self.problem.demands[k]:
                return False
        return True
    
    #How much is not covered yet for a certain customer
    def is_not_covered(self, city, genome, amounts):
        summ = 0
        for i, car in enumerate(genome):
            if city in car.route:
                city_idx = car.route.index(city)
                summ += amounts[i][city_idx]
        return self.problem.demands[city] - summ
            
          
    # Uniform crossover
    def uniform_crossover(self, mating_pool, amounts):
        
        pool = mating_pool.copy()
        amounts_temp = amounts.copy()
        #The offspring
        offSpring = []
        cargoList = []
        
        
        for i in range(self.mating_pool_size):
            
            
            #Choose randomly two parents
            mating_parents = np.random.choice(range(len(pool)), 2, replace=False)
            
            #Create the routes of each parent and its cargo amounts
            parent1_route_cost = [(1, i, car.route_cost/len(car.route)) for i, car in enumerate(pool[mating_parents[0]])]
            parent2_route_cost = [(2, i, car.route_cost/len(car.route)) for i, car in enumerate(pool[mating_parents[1]])]
            
            parent1_amounts = amounts_temp[mating_parents[0]].copy()
            parent2_amounts = amounts_temp[mating_parents[1]].copy()
            
            #List of chosen vehicles
            chosen_routes = np.zeros(len(parent1_route_cost))
            
            #Sort the vehicles from both parents combined based on a fitness function
            merged = sorted(parent1_route_cost + parent2_route_cost, key=lambda x : x[2])
            
            child = [Car(self.problem.capacities[i], self.problem.costs[i], list([]), 0) for i in range(chosen_routes.shape[0])]
            amounts_child = [[] for _ in range(chosen_routes.shape[0])]
            
            vehicle = 0
            
            #while not all customers covered and there is a route that not chosen yet, keep breeding
            while not self.all_covered(child, amounts_child) and any(chosen_routes==0) :
                
                #The best route (vehicle) 
                route_tuple = merged[vehicle]
            
                #if the route is from the first parent 
                if route_tuple[0]==1:
                    parent_car = copy.deepcopy(pool[mating_parents[0]][route_tuple[1]])
                    amountsParent = parent1_amounts[route_tuple[1]]
                
                #if it's from the 2nd parent 
                else:
                    parent_car = copy.deepcopy(pool[mating_parents[1]][route_tuple[1]])
                    amountsParent = parent2_amounts[route_tuple[1]]
                
                #if the vehicle not assigned yet to a route
                if not chosen_routes[route_tuple[1]]:
                    
                    
                    #if all cities in this route not fully covered yet
                    ####if all(np.array([self.is_not_covered(c, child, amounts) for c in genome]) != 0):
                        
                    child[route_tuple[1]] = parent_car
                    amounts_child[route_tuple[1]] += amountsParent

                    chosen_routes[route_tuple[1]] = 1

                    #Check if a customer is overcovered, reduce the last delivery 
                    for c in range(1, len(child[route_tuple[1]].route)-1):
                        
                        if self.is_not_covered(child[route_tuple[1]].route[c], child, amounts_child) < 0:
                            amounts_child[route_tuple[1]][c] += self.is_not_covered(child[route_tuple[1]].route[c], child, amounts_child)
                
                vehicle += 1
                    
                
                ###if all covered break or repair
                
                
            offSpring.append(child)
            cargoList.append(amounts_child)
            
        return offSpring, cargoList


In [17]:

class Mutation(object):
    
    
    
    # parameters needed for recombination in general
    
    def __init__(self, p, problem):
        self.p = p
        self.problem = problem
    
    #Insert a customer in a route the best place based on minimum distances
    def bestInsertion(self, cust, route):
        payoffs = [self.problem.distances[route[i]][route[i+1]]-
                     self.problem.distances[route[i]][cust]-
                     self.problem.distances[cust][route[i+1]] for i in range(len(route[0:-1]))]
        c = payoffs.index(max(payoffs))
        return c   #between c and c+1
    

    #How much is not covered yet for a certain customer
    def is_not_covered(self, city, genome, amounts):
        summ = 0
        for i, car in enumerate(genome):
            if city in car.route:
                city_idx = car.route.index(city)
                summ += amounts[i][city_idx]
        return self.problem.demands[city] - summ
            
    
    #Random resetting mutation
    
    def mutate(self, population, amounts):
        
       
        #Each individuals are exposed to mutations
        for i, indiv in enumerate(population):
            
            #pick a random vehicle
            
            routes = [idx for idx, routee in enumerate(indiv) if len(routee.route)>2]
            
            route_idx = np.random.choice(routes)
            routes.remove(route_idx)
            #pick a random cust on this route
            custIdx = np.random.randint(low=1, high=len(indiv[route_idx].route)-1)
            cust = indiv[route_idx].route[custIdx]
            routeNew = route_idx
            idx = custIdx
            
            #in case of a mutation
            if random.uniform(0,1) < self.p:
                
                while True:
                    
                    #pick a random new route that is different from the old one
                    routeNew = np.random.choice(routes)
                    #The vehicle has to have a space greater than the cargo delivered to the customer
                    #so that this cargo can be delivered by the new vehicle
                    if (self.problem.capacities[routeNew] - sum(amounts[i][routeNew][c] for c in range(len(indiv[routeNew].route)))
                     >= amounts[i][route_idx][custIdx]):
                        idx = self.bestInsertion(cust, indiv[routeNew].route)
                        break
            del indiv[route_idx].route[custIdx]
            del amounts[i][route_idx][custIdx]
            indiv[routeNew].route.insert(idx+1, cust)
            amounts[i][routeNew].insert(idx+1, cust)
            
        
   

In [18]:
class GeneticAlgo(object):
    
    def __init__(self, problem, termination=20, initializer='main_init', selector='roulette wheel', recombiner='uniform crossover', mutator='random resetting', pop_size=50, crossOver_rate=1, mutation_rate=0.25):
        
        self.problem = problem
        self.termination = termination
        self.pop_size = pop_size
        self.initializer = initializer
        self.selector = selector
        self.recombiner = recombiner
        self.mutator = mutator
        self.crossOver_rate = crossOver_rate
        self.mutation_rate = mutation_rate
        self.initialization_dict = {'main_init': Initializer(self.problem, self.pop_size).main_initialization()}
        self.selection_dict = {'roulette wheel': Selector().roulette_wheel_GA}
        self.recombination_dict = {'uniform crossover': Recombiner(self.pop_size, self.problem).uniform_crossover}
        self.mutation_dict = {'random resetting': Mutation(self.mutation_rate, self.problem).mutate}

        
    
    def top_individual(self, population):
        
        return sorted(population, key= lambda x : cost(x))[0]
    
    def order(self, list1, list2):
        return [list2.index(list1[i]) for i in range(len(list1))]
    
    def pop_order(self, pop1, pop2):
        return [[self.order(pop1[sol][car].route, pop2[sol][car].route) for car in range(len(pop1[sol]))] for sol in range(len(pop1))]
    
    def amounts_ordered(self, pop1, pop2, cargoList):
        pop_orderr = self.pop_order(pop1, pop2)
        amountsNew = []
        for sol in range(len(cargoList)):
            amounts = []
            for car in range(len(cargoList[sol])):
                amount = []
                for custOldIdx in pop_orderr[sol][car]:
                    amount.append(cargoList[sol][car][custOldIdx])
                amounts.append(amount)
            amountsNew.append(amounts)
        return amountsNew
            
    
    def evolve(self):
        
        population, amounts = self.initialization_dict[self.initializer]
        oldPop = copy.deepcopy(population)
        
        aco(self.problem.distances, population)
        amounts = self.amounts_ordered(population, oldPop, amounts)
        top_individual = self.top_individual(population)
        
        count = 0
        
        #For monitoring
        top_individuals = [top_individual]
        generationNr = 0
        
        while True :
            count += 1
            
            mating_pool, amounts = self.selection_dict[self.selector](population, amounts, len(population))
            
            offSpring, amounts = self.recombination_dict[self.recombiner](mating_pool, amounts)
            
            self.mutation_dict[self.mutator](offSpring, amounts)
            
           
            population = offSpring
            if generationNr == 7:
                
            
                oldPop = copy.deepcopy(population)
                aco(self.problem.distances, population)
                amounts = self.amounts_ordered(population, oldPop, amounts)
            
            #population = self.replacer_dict[self.replacer](mating_pool, offSpring)
            
            
            #For monitoringq
            top_individuals.append(self.top_individual(population))
            generationNr += 1
            print("generNR: ", generationNr)
            #Termination condition: no improvement in the last 25 generations
            if cost(self.top_individual(population))<cost(top_individual):
                top_individual = self.top_individual(population)
                count = 0
            elif count==self.termination:
                break
                
            print("minimum cost sofar: ", cost(top_individual))
        
        #Return the best individual within the final population
        #return self.top_individual(population)
        return top_individuals, generationNr   

In [ ]:
# Main Program


problem = Problem(distance, capacity, trans_cost, demands, cost)

In [ ]:

ga = GeneticAlgo(problem)
tops, gens = ga.evolve()